In [1]:
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.svm import SVC

In [2]:
iris = load_iris()
dir(iris)

['DESCR',
 'data',
 'data_module',
 'feature_names',
 'filename',
 'frame',
 'target',
 'target_names']

In [8]:
df = pd.DataFrame(iris['data'],columns=iris['feature_names'])
df['flower'] = iris['target']
df['flower'] = df['flower'].apply(lambda x: iris['target_names'][x])
df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),flower
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [11]:
from sklearn.model_selection import train_test_split
X = df.drop(['flower'],axis='columns')
y = df['flower']
X_train , X_test , y_train , y_test = train_test_split(X,y,test_size=0.25)

In [12]:
model = SVC(kernel='rbf',C=30,gamma='auto').fit(X_train,y_train)
model.score(X_test,y_test)

0.9736842105263158

In [15]:
from sklearn.model_selection import cross_val_score

In [21]:
cross_val_score(SVC(kernel='linear',C=10,gamma='auto'),X,y,cv=5)

array([1.        , 1.        , 0.9       , 0.96666667, 1.        ])

In [19]:
cross_val_score(SVC(kernel='rbf',C=10,gamma='auto'),X,y,cv=5)

array([0.96666667, 1.        , 0.96666667, 0.96666667, 1.        ])

In [20]:
cross_val_score(SVC(kernel='rbf',C=20,gamma='auto'),X,y,cv=5)

array([0.96666667, 1.        , 0.9       , 0.96666667, 1.        ])

In [26]:
import numpy as np
kernels = ['rbf','linear']
C = [1,10,20]
avg_scores = {}
for kval in kernels:
    for cval in C:
        cv_scores = cross_val_score(SVC(kernel=kval,C=cval,gamma='auto'),X,y,cv=5)
        avg_scores[kval+'_'+str(cval)] = np.average(cv_scores)
avg_scores

{'rbf_1': 0.9800000000000001,
 'rbf_10': 0.9800000000000001,
 'rbf_20': 0.9666666666666668,
 'linear_1': 0.9800000000000001,
 'linear_10': 0.9733333333333334,
 'linear_20': 0.9666666666666666}

In [38]:
# Because SVC model changes everytime a new sample is taken so we do Cross validation for more stable score
# The GridSearch does the same as the for loop above but way more efficent

In [33]:
from sklearn.model_selection import GridSearchCV

clf = GridSearchCV(
    SVC(gamma='auto'),
    {'C':[1,10,20],  
     'kernel':['rbf','linear'] 
    },
    cv = 5,
    return_train_score = True
)
clf.fit(X,y)
clf.cv_results_

{'mean_fit_time': array([0.00339684, 0.00219431, 0.00219364, 0.00259304, 0.0021944 ,
        0.00259323]),
 'std_fit_time': array([0.00101454, 0.00039868, 0.00039914, 0.00048881, 0.00039911,
        0.00048877]),
 'mean_score_time': array([0.00258741, 0.00179496, 0.00199509, 0.00179524, 0.00159607,
        0.00139642]),
 'std_score_time': array([1.01508207e-03, 3.98945933e-04, 3.01578299e-07, 3.98612318e-04,
        4.89084195e-04, 4.88908721e-04]),
 'param_C': masked_array(data=[1, 1, 10, 10, 20, 20],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_kernel': masked_array(data=['rbf', 'linear', 'rbf', 'linear', 'rbf', 'linear'],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'C': 1, 'kernel': 'rbf'},
  {'C': 1, 'kernel': 'linear'},
  {'C': 10, 'kernel': 'rbf'},
  {'C': 10, 'kernel': 'linear'},
  {'C': 20, 'kernel': 'rbf'},
  {'C': 20

In [36]:
df = pd.DataFrame(clf.cv_results_)
df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,...,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
0,0.003397,0.001015,0.002587,1.015082e-03,1,rbf,"{'C': 1, 'kernel': 'rbf'}",0.966667,1.0,0.966667,...,0.980000,0.016330,1,0.983333,0.983333,0.991667,0.983333,0.975000,0.983333,0.005270
1,0.002194,0.000399,0.001795,3.989459e-04,1,linear,"{'C': 1, 'kernel': 'linear'}",0.966667,1.0,0.966667,...,0.980000,0.016330,1,0.975000,0.975000,0.991667,0.983333,0.983333,0.981667,0.006236
2,0.002194,0.000399,0.001995,3.015783e-07,10,rbf,"{'C': 10, 'kernel': 'rbf'}",0.966667,1.0,0.966667,...,0.980000,0.016330,1,0.975000,0.966667,0.991667,0.983333,0.966667,0.976667,0.009718
3,0.002593,0.000489,0.001795,3.986123e-04,10,linear,"{'C': 10, 'kernel': 'linear'}",1.000000,1.0,0.900000,...,0.973333,0.038873,4,0.966667,0.966667,0.991667,0.991667,0.975000,0.978333,0.011304
4,0.002194,0.000399,0.001596,4.890842e-04,20,rbf,"{'C': 20, 'kernel': 'rbf'}",0.966667,1.0,0.900000,...,0.966667,0.036515,5,0.966667,0.975000,0.991667,0.991667,0.975000,0.980000,0.010000
5,0.002593,0.000489,0.001396,4.889087e-04,20,linear,"{'C': 20, 'kernel': 'linear'}",1.000000,1.0,0.900000,...,0.966667,0.042164,6,0.975000,0.966667,0.991667,1.000000,0.975000,0.981667,0.012247


In [37]:
df[['param_C','param_kernel','mean_test_score']].bds()

,param_C,param_kernel,mean_test_score
0,1,rbf,0.980000
1,1,linear,0.980000
2,10,rbf,0.980000
3,10,linear,0.973333
4,20,rbf,0.966667
5,20,linear,0.966667


In [39]:
dir(clf)

['__abstractmethods__',
 '__annotations__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__sklearn_clone__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_build_request_for_signature',
 '_check_feature_names',
 '_check_n_features',
 '_check_refit_for_multimetric',
 '_estimator_type',
 '_format_results',
 '_get_default_requests',
 '_get_metadata_request',
 '_get_param_names',
 '_get_tags',
 '_more_tags',
 '_parameter_constraints',
 '_repr_html_',
 '_repr_html_inner',
 '_repr_mimebundle_',
 '_required_parameters',
 '_run_search',
 '_select_best_index',
 '_validate_data',
 '_validate_params',
 'best_estimator_',
 'best_index_',
 'best_params_',
 'best_score_',
 '

In [40]:
clf.best_score_

0.9800000000000001

In [41]:
clf.best_params_ # best parameters

{'C': 1, 'kernel': 'rbf'}

In [55]:
from sklearn.model_selection import RandomizedSearchCV # same as gridsearch but here it take the parameters in random
rs = RandomizedSearchCV(SVC(gamma='auto'),
     {
     'C' :[1,10,20],
     'kernel': ['rbf','linear']
     },
     cv=5,
     n_iter=2 # only try 2 combinations
     )
rs.fit(X,y)

RandomizedSearchCV(cv=5, estimator=SVC(gamma='auto'), n_iter=2,
                   param_distributions={'C': [1, 10, 20],
                                        'kernel': ['rbf', 'linear']})

In [56]:
pd.DataFrame(rs.cv_results_)[['param_C','param_kernel','mean_test_score']]

,param_C,param_kernel,mean_test_score
0,1,rbf,0.980000
1,20,rbf,0.966667


In [57]:
#---How to choose best model----#

In [58]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

In [61]:
model_params = {
    'svm': {
        'model': SVC(gamma='auto'),
        'params' : {
            'C': [1,10,20],
            'kernel': ['rbf','linear']
        }  
    },
    'random_forest': {
        'model': RandomForestClassifier(),
        'params' : {
            'n_estimators': [1,5,10]
        }
    },
    'logistic_regression' : {
        'model': LogisticRegression(solver='liblinear',multi_class='auto'),
        'params': {
            'C': [1,5,10]
        }
    }
}

In [62]:
model_params.items()

dict_items([('svm', {'model': SVC(gamma='auto'), 'params': {'C': [1, 10, 20], 'kernel': ['rbf', 'linear']}}), ('random_forest', {'model': RandomForestClassifier(), 'params': {'n_estimators': [1, 5, 10]}}), ('logistic_regression', {'model': LogisticRegression(solver='liblinear'), 'params': {'C': [1, 5, 10]}})])

In [63]:
scores = []

for model_name, mp in model_params.items():
    clf =  GridSearchCV(mp['model'], mp['params'], cv=5, return_train_score=False)
    clf.fit(iris.data, iris.target)
    scores.append({
        'model': model_name,
        'best_score': clf.best_score_,
        'best_params': clf.best_params_
    })
    
df = pd.DataFrame(scores,columns=['model','best_score','best_params'])
df

,model,best_score,best_params
0,svm,0.980000,"{'C': 1, 'kernel': 'rbf'}"
1,random_forest,0.966667,{'n_estimators': 1}
2,logistic_regression,0.966667,{'C': 5}


In [66]:
# Use Digits Dataset and find the best hyperparameters and model

from sklearn import svm
from sklearn.datasets import load_digits
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier

In [68]:
digits = load_digits()
dir(digits)

['DESCR', 'data', 'feature_names', 'frame', 'images', 'target', 'target_names']

In [75]:
df = pd.DataFrame(digits['data'])
df.head()

,0,1,2,3,4,5,6,7,8,9,...,54,55,56,57,58,59,60,61,62,63
0,0.0,0.0,5.0,13.0,9.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,6.0,13.0,10.0,0.0,0.0,0.0
1,0.0,0.0,0.0,12.0,13.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,11.0,16.0,10.0,0.0,0.0
2,0.0,0.0,0.0,4.0,15.0,12.0,0.0,0.0,0.0,0.0,...,5.0,0.0,0.0,0.0,0.0,3.0,11.0,16.0,9.0,0.0
3,0.0,0.0,7.0,15.0,13.0,1.0,0.0,0.0,0.0,8.0,...,9.0,0.0,0.0,0.0,7.0,13.0,13.0,9.0,0.0,0.0
4,0.0,0.0,0.0,1.0,11.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,2.0,16.0,4.0,0.0,0.0


In [78]:
df['target'] = digits['target']
df.head(1)

,0,1,2,3,4,5,6,7,8,9,...,55,56,57,58,59,60,61,62,63,target
0,0.0,0.0,5.0,13.0,9.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,6.0,13.0,10.0,0.0,0.0,0.0,0


In [80]:
X = df.drop(['target'],axis='columns')
y = df['target']
X_train , X_test , y_train , y_test = train_test_split(X,y,test_size=0.25)

In [90]:
model_params = {
    'svm': {
        'model': SVC(gamma='auto'),
        'params' : {
            'C': [1,10,20],
            'kernel': ['rbf','linear']
        }  
    },
    'random_forest': {
        'model': RandomForestClassifier(),
        'params' : {
            'n_estimators': [1,5,10]
        }
    },
    'logistic_regression' : {
        'model': LogisticRegression(solver='liblinear',multi_class='auto'),
        'params': {
            'C': [1,5,10]
        }
    },
    'decision_tree': {
        'model' : DecisionTreeClassifier(),
        'params' : {
            'criterion' : ["gini", "entropy", "log_loss"]
        }
    },
    'GaussianNB': {
            'model' : GaussianNB(),
            'params' :{
            }
    },
    'MultinoimalNB' : {
            'model' : MultinomialNB(),
        'params' :{
            }
    }
}

In [91]:
model_params.values()

dict_values([{'model': SVC(gamma='auto'), 'params': {'C': [1, 10, 20], 'kernel': ['rbf', 'linear']}}, {'model': RandomForestClassifier(), 'params': {'n_estimators': [1, 5, 10]}}, {'model': LogisticRegression(solver='liblinear'), 'params': {'C': [1, 5, 10]}}, {'model': DecisionTreeClassifier(), 'params': {'criterion': ['gini', 'entropy', 'log_loss']}}, {'model': GaussianNB(), 'params': {}}, {'model': MultinomialNB(), 'params': {}}])

In [92]:
scores = []

for model_name,mp in model_params.items():
    clf = GridSearchCV(mp['model'],mp['params'],cv=5)
    clf.fit(X,y)
    scores.append({
        'model':model_name,
        'best_score' : clf.best_score_,
        'best_params': clf.best_params_
    })

In [97]:
df = pd.DataFrame(scores)
df

,model,best_score,best_params
0,svm,0.947697,"{'C': 1, 'kernel': 'linear'}"
1,random_forest,0.905981,{'n_estimators': 10}
2,logistic_regression,0.922114,{'C': 1}
3,decision_tree,0.816948,{'criterion': 'log_loss'}
4,GaussianNB,0.806928,{}
5,MultinoimalNB,0.870350,{}
